In [21]:
import json
from web3 import Web3
from taxes_abi import *

In [22]:
ganache_url = 'HTTP://127.0.0.1:7545'

In [23]:
web3 = Web3(Web3.HTTPProvider(ganache_url))

In [24]:
web3.eth.defaultAccount = web3.eth.accounts[0] 
print(web3.isConnected())

True


In [25]:
contract_abi = json.loads(abi)
contract_bytecode = bytecode[0]

In [26]:
def deploy_contract(abi, bytecode):
    contract = web3.eth.contract(abi=abi, bytecode=bytecode)
    tx_hash = contract.constructor().transact()
    tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
    return tx_receipt.contractAddress


In [27]:
contract_address = deploy_contract(abi=contract_abi, bytecode=contract_bytecode)

print(contract_address)

0xBeFA3a8C009e2A229C594c094279c8C6f16F50e5


In [12]:
import time

def handle_event(event):
    print(event)

def log_loop(event_filter, poll_interval):
    while True:
        for event in event_filter.get_new_entries():
            handle_event(event)
        time.sleep(poll_interval)

def main():
    block_filter = web3.eth.filter('latest')
    log_loop(block_filter, 2)

In [13]:
main()

b'\x94N\xd7\xf91\xaau\x89\xff\xe6\xe1m\xe4\xa1\xeb\xa1\xddU\xf2\xc7\x85\x07\xd9b\x076\xdf\xa7\xb9\xf6F('
b'\x14\xe3\x0f\xd5c\xc0)>\xb1\xa8%\r\xee\xd7\xdd\xd6\x82I\xfc\xc2k\xf1-\xa8\xb1\x9d;\x16\xfb\xb5\xe3-'


KeyboardInterrupt: 